<a href="https://colab.research.google.com/github/laskari/END-Program/blob/main/Week-9/Model_1_AM_QADataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
print('Train Data')
data_train = pd.read_json('/content/drive/MyDrive/END School of AI/Datasets/END Datasets/ambignq/train.json', lines=False, orient=str)
data_new_train= data_train[['question','nq_answer']]
data_new_train['question'] = [str(question) for question in data_train['question']]
data_new_train['nq_answer'] = [str(answer) for answer in data_train['nq_answer']]

print('Validation Data')

data_dev = pd.read_json('/content/drive/MyDrive/END School of AI/Datasets/END Datasets/ambignq/dev.json', lines=False, orient=str)
data_dev.head(3)
data_new_dev = data_dev[['question','nq_answer']]
data_new_dev['question'] = [str(question) for question in data_dev['question']]
data_new_dev['nq_answer'] = [str(answer) for answer in data_dev['nq_answer']]

Train Data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Validation Data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
data_new_train['nq_answer']= data_new_train['nq_answer'].apply(lambda x: x.strip('[]'))
data_new_dev['nq_answer']= data_new_dev['nq_answer'].apply(lambda x: x.strip('[]'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
data_new_dev

,question,nq_answer
0,Who plays the doctor in dexter season 1?,'Tony Goldwyn'
1,How often does spermatogeneis—the production o...,'74 days'
2,When was the first remote control tv invented?,'1950'
3,Why did the st louis cardinals move to arizona?,'1988'
4,How many episodes are in season 2 of chesapeak...,'10'
...,...,...
1997,How long is a rainbow six siege game?,'a few minutes'
1998,Where did the free settlers settle in australia?,'Liberty Plains'
1999,Real name of gwen stacy in amazing spiderman?,'Emma Stone'
2000,Who owns the biggest house in the united states?,'William A.V. Cecil'


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import random
from torchtext import data 
import math
import time

In [8]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

spacy_en = spacy.load('en')

In [9]:
def tokenize_en(text):
  return [tok.text for tok in spacy_en.tokenizer(text)]

In [10]:
Question = Field(tokenize = 'spacy',
            init_token='<sos>',
            eos_token ='<eos',
            lower = True)

Sentence = Field(tokenize = 'spacy',
            init_token='<sos>',
            eos_token ='<eos',
            lower = True)

fields = [('Question', Question),('Sentence',Sentence)]


In [11]:
example_train = [data.Example.fromlist([data_new_train.question[i],data_new_train.nq_answer[i]], fields) for i in range(data_new_train.shape[0])] 

example_dev = [data.Example.fromlist([data_new_dev.question[i],data_new_dev.nq_answer[i]], fields) for i in range(data_new_dev.shape[0])]

In [12]:
vars(example_train[1])

{'Question': ['who',
  'played',
  'george',
  'washington',
  'in',
  'the',
  'john',
  'adams',
  'series',
  '?'],
 'Sentence': ["'", 'david', 'morse', "'"]}

In [13]:
vars(example_dev[1])

{'Question': ['how',
  'often',
  'does',
  'spermatogeneis',
  '—',
  'the',
  'production',
  'of',
  'sperm',
  '—',
  'occur',
  '?'],
 'Sentence': ["'", '74', 'days', "'"]}

In [14]:
train_Dataset = data.Dataset(example_train, fields)
dev_Dataset = data.Dataset(example_dev, fields)
print(len(train_Dataset))
print(len(dev_Dataset))

10036
2002


In [15]:
Question.build_vocab(train_Dataset)
Sentence.build_vocab(train_Dataset)

In [16]:
print('Size of input vocab : ', len(Question.vocab))
print('Size of label vocab : ', len(Sentence.vocab))

Size of input vocab :  10024
Size of label vocab :  9643


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [18]:
train_iterator, valid_iterator = data.BucketIterator.splits((train_Dataset, dev_Dataset), batch_size = 32, sort=False, device = device)

In [19]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [20]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [21]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [22]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Sentence.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [23]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10024, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(9643, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=9643, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,539,307 trainable parameters


In [25]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = Sentence.vocab.stoi[Sentence.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [26]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Sentence
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            trg = batch.Sentence

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [28]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [29]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 26s
	Train Loss: 4.802 | Train PPL: 121.694
	 Val. Loss: 4.471 |  Val. PPL:  87.472
Epoch: 02 | Time: 0m 25s
	Train Loss: 3.928 | Train PPL:  50.825
	 Val. Loss: 4.369 |  Val. PPL:  78.949
Epoch: 03 | Time: 0m 25s
	Train Loss: 3.633 | Train PPL:  37.830
	 Val. Loss: 4.353 |  Val. PPL:  77.745
Epoch: 04 | Time: 0m 25s
	Train Loss: 3.418 | Train PPL:  30.507
	 Val. Loss: 4.361 |  Val. PPL:  78.314
Epoch: 05 | Time: 0m 25s
	Train Loss: 3.238 | Train PPL:  25.494
	 Val. Loss: 4.470 |  Val. PPL:  87.368
Epoch: 06 | Time: 0m 25s
	Train Loss: 3.068 | Train PPL:  21.501
	 Val. Loss: 4.493 |  Val. PPL:  89.399
Epoch: 07 | Time: 0m 25s
	Train Loss: 2.878 | Train PPL:  17.776
	 Val. Loss: 4.617 |  Val. PPL: 101.183
Epoch: 08 | Time: 0m 25s
	Train Loss: 2.675 | Train PPL:  14.516
	 Val. Loss: 4.729 |  Val. PPL: 113.156
Epoch: 09 | Time: 0m 25s
	Train Loss: 2.395 | Train PPL:  10.967
	 Val. Loss: 4.982 |  Val. PPL: 145.717
Epoch: 10 | Time: 0m 26s
	Train Loss: 2.048 | Train PPL